In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_info_columns', 10000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)


In [ ]:

def get_dataframe_summary(df):
    """
    Returns a summary DataFrame for the given DataFrame.
    
    The summary includes:
      - Data Type
      - Non Null Count
      - Null Count
      - Null Percentage
      - Unique Values count
    """
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    summary_df = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non Null Count': df.count(),
        'Null Count': df.isna().sum(),
        'Null Percentage': (df.isna().sum() / len(df) * 100).round(2),
        'Unique Values': [df[col].nunique() for col in df.columns],
    })
    
    return summary_df

In [ ]:
from pathlib import Path
import sys

# 1) Where is this notebook?
notebook_dir = Path.cwd()

# 2) Climb up until you get to the folder that contains "app/"
#    parents[2] goes up from objetivo_2 → notebooks → objetivos → …
#    count how many levels from objetivo_2 to BOTS_RPA: in your case it's 8 levels
project_root = notebook_dir.parents[8]

# 3) Insert it at front of sys.path
sys.path.insert(0, str(project_root))

# 4) Now imports of "app.…" will succeed

from app.modules.sga.minpub.report_validator.service.objetivos.calculations import extract_tecnico_reports_without_hours_last_dates
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_df_word_datos_informe_tecnico
from app.modules.sga.minpub.report_validator.service.objetivos.preprocessing import preprocess_df_word_telefonia_informe_tecnico
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_datos_informe_corte_excel import merge_word_datos_informe_corte_excel 
from app.modules.sga.minpub.report_validator.service.objetivos.mergers.merge_word_telefonia_informe_corte_excel import merge_word_telefonia_informe_corte_excel
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.o2_informe_tecnico_validator import validate_informe_tecnico_word
from app.modules.sga.minpub.report_validator.service.objetivos.objetivo_2.o2_informe_tecnico_validator import build_failure_messages_validate_informe_tecnico_word


In [ ]:
BASE_DIR = Path.cwd().parent.parent.parent.parent.parent.parent.parent.parent.parent
SAVE_DIR_EXTRACT_EXCEL = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "excel"/ "CORTE 2.xlsx"
SAVE_DIR_EXTRACT_SGA_335 = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sga_335" / "sga_reporte_30-03-2025_06-04-2025_20250410_173936.xlsx"
CID_CUISMP_PATH = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "sharepoint_cid_cuismp" / "MINPU - CID-CUISMP - AB.xlsx"
DIR_PARADAS_RELOJ = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "pausa_cliente" / "sga_reporte_30-03-2025_06-04-2025_20250428_151430.xlsx"
DIR_WORD_DATOS = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_datos" / "COMPONENTE 2-DATOS - CORTE 2.docx"
DIR_WORD_TELEFONIA = BASE_DIR / "media" / "minpub" / "validator_report" / "extract" / "word_telefonia" / "COMPONENTE 4 - TELEFONOS - CORTE 2.docx"



In [ ]:
df_corte_excel = pd.read_excel(SAVE_DIR_EXTRACT_EXCEL, skipfooter=2, engine="openpyxl")
df_sga_dinamico_335 = pd.read_excel(SAVE_DIR_EXTRACT_SGA_335) 
df_sga_dinamico_380 = pd.read_excel(DIR_PARADAS_RELOJ)
df_cid_cuismp_sharepoint = pd.read_excel(CID_CUISMP_PATH)

df_word_datos_informe_tec =  extract_tecnico_reports_without_hours_last_dates(DIR_WORD_DATOS)
df_word_telefonia_informe_tec = extract_tecnico_reports_without_hours_last_dates(DIR_WORD_TELEFONIA)




In [ ]:

df_process_word = preprocess_df_word_datos_informe_tecnico(df_word_datos_informe_tec)
summary = get_dataframe_summary(df_process_word)
#summary

In [ ]:

df_corte_excel = preprocess_corte_excel(df_corte_excel)



In [ ]:
df_matched_word_datos_informe_tecnico_corte_excel = merge_word_datos_informe_corte_excel(
        df_word_datos_informe_tec,
        df_corte_excel,
        'both'
        )
    
    #INFORME TECNICO - TELEFONIA - EXCEL
df_matched_word_telefonia_informe_tecnico_corte_excel = merge_word_telefonia_informe_corte_excel(
        df_word_telefonia_informe_tec,
        df_corte_excel,
        'both'
        )

In [ ]:
df_vali = validate_informe_tecnico_word(df_matched_word_datos_informe_tecnico_corte_excel, 'COMPONENTE II')
df_vali_tel = validate_informe_tecnico_word(df_matched_word_telefonia_informe_tecnico_corte_excel, 'COMPONENTE IV' )


In [ ]:
df_mess = build_failure_messages_validate_informe_tecnico_word(df_vali)
#df_mess_TELE = build_failure_messages_validate_informe_tecnico_word(df_vali_tel)
df_mess